# Exploratory Data Analysis
### Vehicles Data

## Introduction

This project aims to develop and deploy a web application to a cloud service, making it accessible to the public. The application will be based on a dataset containing information about car sales advertisements. This part of the project contains the Exploratory Data Analysis (EDA), which is crucial for understanding the data, clean and preprocess, and identify patterns and trends.

## 1. Data Overview

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
vehicles_data = pd.read_csv("../vehicles_us.csv")

In [ ]:
vehicles_data.head(10)

In [ ]:
vehicles_data.info()

In [ ]:
vehicles_data.shape

### 1.1 Conclusion
Every column name in the dataframe has an appropiate format. No adjustment is required.

There appear to be missing values in the following columns: model_year, cylinders, odometer, paint_color and is_4wd. The NaN values of the latter column seem to represent `False` (indicating the vehicle is not 4wd). We will explore this further. Some columns require a change of data type.

To proceed, we need to preprocess the data. 

## 2. Data Preprocessing

We want to start by checking the unique values for the `is_4wd` column.

In [ ]:
vehicles_data['is_4wd'].unique()

Since the only two available values are `1` and `nan`, we will change the `nan`s to `0` and keep this column as numeric. Therefore, `1 = True` and `0 = False`.

In [ ]:
vehicles_data.loc[vehicles_data['is_4wd'].isna(), ['is_4wd']] = 0
vehicles_data['is_4wd'].unique()

In [ ]:
vehicles_data['is_4wd'] = pd.to_numeric(vehicles_data['is_4wd']).astype(int)

Before proceeding with changing that data types as required, we want to check the duplicates and adjust them as necessary.

In [ ]:
duplicates = vehicles_data.duplicated().sum()
duplicates

No duplicates were found.

Although some columns such as `model_year` and `cylinders` are numeric, they are treated as categorical data. To explicitly indicate that they are categories rather than numerical values, we will convert these columns to strings (objects). We'll first convert them to numeric to remove the decimal part and then to strings. Additionally, we will fill the null values of `model_year`, `cylinders`, and `paint_color` with `not specified`.

In [ ]:
vehicles_data['model_year'] = pd.to_numeric(vehicles_data['model_year'], errors='coerce')
vehicles_data['model_year'] = vehicles_data['model_year'].astype('Int64')

vehicles_data['cylinders'] = pd.to_numeric(vehicles_data['cylinders'], errors='coerce')
vehicles_data['cylinders'] = vehicles_data['cylinders'].astype('Int64')

vehicles_data.info()

In [ ]:
# using -1 as a placeholder to fill the null values for numeric columns
vehicles_data['model_year'] = vehicles_data['model_year'].fillna(value=-1)
vehicles_data['cylinders'] = vehicles_data['cylinders'].fillna(value=-1)

vehicles_data['paint_color'] = vehicles_data['paint_color'].fillna(value='not specified')

In [ ]:
vehicles_data['model_year'] = vehicles_data['model_year'].astype(str)
vehicles_data['cylinders'] = vehicles_data['cylinders'].astype(str)

vehicles_data['model_year'] = vehicles_data['model_year'].replace('-1', 'not specified')
vehicles_data['cylinders'] = vehicles_data['cylinders'].replace('-1', 'not specified')

Likewise, we'll treat the `date_posted` column as a date.

In [ ]:
vehicles_data['date_posted'] = pd.to_datetime(vehicles_data['date_posted'], format ="%Y-%m-%d")
vehicles_data.info()

In [ ]:
vehicles_data.head(10)

### 2.1 Conclusion



## 3. EDA

We will initiate by adding a new column `make` derived from the column `model` and a new column `month` from the column `date_posted`. This will enable us to conduct a general review of the data. 

In [ ]:
vehicles_data['make'] = vehicles_data['model'].str.split().str[0]
vehicles_data['month'] = vehicles_data['date_posted'].dt.strftime('%y-%m')

In [ ]:
# checking unique values to verify that the names of all makes in the data set are one word
vehicles_data['make'].unique()

In [ ]:
# checking the unique values for the month column
vehicles_data['month'].unique()

In [ ]:
# saving the clean dataframe to csv file.
# This file will be used in the web app.

#vehicles_data.to_csv('vehicles_clean.csv', index=False)

### 3.1 Average Prices and Distribution

We'll proceed to create some visualizations to identify general trends in our data. We'll start by examining the distribution of prices and the average price per make.

In [ ]:
px.histogram(vehicles_data, x='price', nbins=100)

To identify outliers in the prices, we will calculate the bounds for outliers. Since the interquartile range method may yield a negative lower bound, which isn't meaningful for prices, and to keep the process simple, we will consider outliers to be values falling below the 5th percentile and above the 95th percentile.

In [ ]:
lower_bound = vehicles_data['price'].quantile(0.05)
upper_bound = vehicles_data['price'].quantile(0.95)

print(f'Lower bound: {lower_bound},  Upper bound: {upper_bound}')

In [ ]:
vehicles_data_upd = vehicles_data[(vehicles_data['price'] < upper_bound) & (vehicles_data['price'] > lower_bound)]

px.histogram(vehicles_data_upd, x='price', nbins=50)

Excluding the outliers, the most frequent prices for the vehicles in our dataset range between \\$3000 and \\$7000.

Let's evaluate the average prices by make.

In [ ]:
average_price = vehicles_data.groupby('make')['price'].mean().reset_index()
average_price = average_price.sort_values(by='price', ascending=False)

In [ ]:
px.bar(average_price, x='make', y='price', height=450)

Mercendes-Benz is clearly the make with higher average prices. Let's dig in a bit more:

In [ ]:
mercedes_data = vehicles_data[vehicles_data['make']=='mercedes-benz']
mercedes_data.head()

In [ ]:
mercedes_by_year = mercedes_data.groupby('model_year')['price'].mean().reset_index()

counts = mercedes_data.groupby('model_year').size().reset_index(name='count')
mercedes_by_year = mercedes_by_year.merge(counts, on='model_year')

mercedes_by_year

It seems that it is the same car has been posted several times, just with the distinction of posting with and without the year and color. Out of 41 posts in total, 39 appear to be duplicates. However, this cannot be stated with certainty, as the odometer in each entry is missing.

In [ ]:
columns = ['price', 'model', 'condition', 'fuel',
       'odometer', 'transmission', 'paint_color']
mercedes_reduced = mercedes_data[columns].copy()
mercedes_reduced.head(10)

In [ ]:
duplicates = mercedes_reduced.duplicated().sum()
duplicates

This means that the counts by any dimensions (e.g. count of posting by make or by condition) might not be accurate, as the same vehicles might have been posted several times.

### 3.2 Distribution of Condition

Taking that into consideration that the count does not necessarily represent the number of distinct vehicles posted, let's explore distribution of conditions by make.

In [ ]:
condition_by_make = vehicles_data.groupby(['make', 'condition'])['condition'].count().unstack(fill_value=0)

condition_by_make

In [ ]:
px.bar(condition_by_make, barmode='group')

We can see that Ford, Chevrolet and Toyota are the ones with most postings, in each case being 'excellent' condition the most predominant. This seems to be true for every make.

### 3.3 Fuel, type and color

We want to explore the number of postings by different categories: fuel, type and color

In [ ]:
by_fuel = vehicles_data.groupby('fuel').size().reset_index(name='count')
by_fuel = by_fuel.sort_values(by='count').reset_index(drop=True)

px.bar(by_fuel, x='count', y='fuel', orientation='h')

In [ ]:
by_type = vehicles_data.groupby('type').size().reset_index(name='count')
by_type = by_type.sort_values(by='count').reset_index(drop=True)

px.bar(by_type, x='count', y='type', orientation='h')

In [ ]:
by_color = vehicles_data.groupby('paint_color').size().reset_index(name='count')
by_color = by_color.sort_values(by='count')

px.bar(by_color, x='count', y='paint_color', orientation='h')

### 3.4 Price by date posted

Let's explore whether there's any relationship between the listing duration and the price.

In [ ]:
px.scatter(vehicles_data, x='price', y='days_listed')

Excluding the outliers as determined before:

In [ ]:
px.scatter(vehicles_data_upd, x='price', y='days_listed',  hover_name='make', hover_data='condition', color='condition')

There doesn't seem to be any relationship between the days listed and the price. It's worth noting that the same vehicles listed a long time ago might have been reposted, and therefore, they appear multiple times in the data.

### 3.5 Listings by date

Lastly, let's explore the numer of listing by date

In [ ]:
by_date = vehicles_data.groupby('date_posted').size().reset_index(name='count')

by_date = by_date.sort_values(by='date_posted')
by_date.head()

In [ ]:
px.line(by_date, x='date_posted',y='count')

In [ ]:
by_month = vehicles_data.groupby('month').size().reset_index(name='count')

by_month = by_month.sort_values(by='month')
by_month.head()

In [ ]:
px.line(by_month, x='month',y='count')

The number of postings appears to be relatively consistent throughout the months, with a few minor peaks that do not seem to be significant. This is with the exception of the last month in the dataset, as it does not contain data for the entire month

### 3.6 Conclusion

The dataset reveals patterns where the most frequent prices fall within the \\$3000 and \\$7000 range, after excluding for outliers. Mercedes-Benz emerges as the make with the highest average prices. However, closer examination reveals that different post might refer to the same vehicles. 

We identified that gas is the predominant mode of fuel and white is the most predominant color. Additionally, we saw that SUV, trucks and sedan account for around 70% of the listings. 

There appears to be no relationship between the number of days listed and the price, likely due to the reposting of vehicles over time. Furthermore, the number of postings remains relatively consistent throughout most months.

While this dataset offers insights into vehicle listings, the count of postings may not accurately represent the number of distint vehicles listed. It is recommended that this dataset is not used for exploring trends such as most posted makes, most frequent colors posted or similar, as it might not accurately reflect the reality. 